<a href="https://colab.research.google.com/github/yashboura303/House_Prediction/blob/master/House_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Richer syntax highlighting

Improved support for nested languages:

```notebook-python
df = pd.io.gbq.read_gbq('''
  SELECT 
    REGEXP_EXTRACT(name, '[a-zA-Z]+'),
    SUM(number) as count
  FROM `bigquery-public-data.usa_names.usa_1910_2013`
  WHERE state = 'TX'
  GROUP BY name
  ORDER BY count DESC
  LIMIT 100
''')
```

In [0]:
from bs4 import BeautifulSoup
import requests
import csv
l = []
def get_lat(strr):
	lat = strr.split(';')[1][16:].split(',')[0]
	lat = lat[1:len(lat)-1]
	return lat
def get_long(strr):
	long = strr.split(';')[1][16:].split(',')[1]
	long = long[1:len(long)-1]
	return long
with open('data_set.csv', 'w', newline='') as csvfile:
	fieldnames = [ 'carpet_area', 'bedrooms', 'bathrooms', 'car_parking', 'furnishing', 'transaction_type','floor', 'overlooking', 'price', 'lat', 'long']
	writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
	writer.writeheader()
	for i in range(10):
		source = requests.get(f'https://www.magicbricks.com/flats-in-mumbai-for-sale-pppfs/page-{i}').text
		soup = BeautifulSoup(source, 'lxml')
		for div in soup.find_all("div", class_='flex relative clearfix m-srp-card__container'):
			dic ={}
			if div.find("span", class_="m-srp-card__price"):
				dic['price']=div.find("span", class_="m-srp-card__price").text
			else:
				continue
			if div.find("span", class_="m-srp-card__title__bhk"):
				dic['bedrooms']=div.find("span", class_="m-srp-card__title__bhk").text.split()[0]
			if div.find("div", class_='m-srp-card__link m-srp-card__link--nearby'):
				if get_lat(div.find("div", class_='m-srp-card__link m-srp-card__link--nearby').get('onclick')) == '0.0':
					continue
				else:
					dic['lat'] = get_lat(div.find("div", class_='m-srp-card__link m-srp-card__link--nearby').get('onclick'))
					dic['long'] = get_long(div.find("div", class_='m-srp-card__link m-srp-card__link--nearby').get('onclick'))
			else:
				continue
					
			dic["car_parking"] = 0
			for div in (div.find_all("div", class_="m-srp-card__summary__item")):
				if div.find("div", class_="m-srp-card__summary__title").text == "car parking":
					main_string = div.find("div", class_="m-srp-card__summary__info").text.strip()
					if main_string:
						if len(main_string) > 10:
							dic['car_parking']=int(main_string.split(",")[0].split()[0]) + int(main_string.split(",")[1].split()[0])
						elif len(main_string) <= 10 and len(main_string)>1:
							dic["car_parking"] = int(main_string.split(",")[0].split()[0])
				if div.find("div", class_="m-srp-card__summary__title").text == "bathroom":
					dic['bathrooms'] = div.find("div", class_="m-srp-card__summary__info").text
				if div.find("div", class_="m-srp-card__summary__title").text == "furnishing":
					main_string = div.find("div", class_="m-srp-card__summary__info").text
					if main_string == "Furnished":
						dic['furnishing'] = 3
					elif main_string == "Unfurnished":
						dic['furnishing'] = 1
					else:
						dic['furnishing'] = 2
				if div.find("div", class_="m-srp-card__summary__title").text == "transaction":
					main_string = div.find("div", class_="m-srp-card__summary__info").text
					if main_string == "Resale":
						dic['transaction_type'] = 0
					else:
						dic['transaction_type'] = 1
				if div.find("div", class_="m-srp-card__summary__title").text == "super area":
					dic['carpet_area'] = int(div.find("div", class_="m-srp-card__summary__info").text.split()[0])/1.25
				if div.find("div", class_="m-srp-card__summary__title").text == "carpet area":
					dic['carpet_area'] = div.find("div", class_="m-srp-card__summary__info").text.split()[0]
				if div.find("div", class_="m-srp-card__summary__title").text == "floor":
					dic['floor'] = div.find("div", class_="m-srp-card__summary__info").text
				if div.find("div", class_="m-srp-card__summary__title").text == "overlooking":
					dic['overlooking'] = div.find("div", class_="m-srp-card__summary__info").text
				
			writer.writerow(dic)



# New Section